In [51]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [52]:
from glob import glob

In [53]:
# create data frame from the labels csv
df = pd.read_csv('./labeling/image_labels.csv')

In [54]:
df.head(3)

,filepath,xmin,xmax,ymin,ymax
0,./images/N1.xml,1093,1396,645,727
1,./images/N100.xml,134,301,312,350
2,./images/N101.xml,31,139,128,161


In [55]:
imgs_path = glob('./labeling/images/*.jpeg')

In [56]:
imgs_path.sort()

In [57]:
df['imgpath'] = imgs_path

<h1><b>Data Preprocessing</b></h1>

In [19]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [25]:
labels_array = df.iloc[:,1:].values

In [45]:
def preprocessing_img(path): 
    load_image = load_img(path,target_size=(244,244))
    load_image_arr = img_to_array(load_image) 
    return load_image_arr

In [68]:
def normalize_img(img, img_label, load_image_arr):
    
    norm_load_image_arr = load_image_arr/255.0
    
    img_arr = cv2.imread(img)
    h, w, d = img_arr.shape
    
    xmin, xmax, ymin, ymax = img_label
    nxmin, nxmax, nymin, nymax = xmin/w, xmax/w, ymin/h, ymax/h
    
    norm_img_label = (nxmin, nxmax, nymin, nymax)
    
    return norm_load_image_arr, norm_img_label

In [71]:
data = []
labels_norm = []

for ind in range(len(df)):
    img_path = df['imgpath'][ind]
    img_label = labels_array[ind]
    
    load_img_arr = preprocessing_img(img_path)
    
    norm_load_image_arr, norm_img_label = normalize_img(img_path, img_label, load_img_arr)
    
    data.append(norm_load_image_arr)
    labels_norm.append(norm_img_label)
    
    

In [74]:
df['norm_load_img_arr'] = data

In [76]:
df['labels_norm'] = labels_norm

<h1>Split train and test</h1>

In [95]:
X = np.array(list(df['norm_load_img_arr']),dtype=np.float32)
y = np.array(list(df['labels_norm']),dtype=np.float32)

In [97]:
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, random_state=42)

<h1>Deep Learning Model</h1>

In [98]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model

import tensorflow as tf